In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/roberta-with-para/logits_train.npy
/kaggle/input/roberta-with-para/logits_test.npy
/kaggle/input/roberta-with-para/logits_val.npy
/kaggle/input/roberta-with-para/runs/Apr22_21-20-40_384de647d8bc/events.out.tfevents.1713820840.384de647d8bc.24.0
/kaggle/input/roberta-with-para/checkpoint-480/config.json
/kaggle/input/roberta-with-para/checkpoint-480/trainer_state.json
/kaggle/input/roberta-with-para/checkpoint-480/training_args.bin
/kaggle/input/roberta-with-para/checkpoint-480/scheduler.pt
/kaggle/input/roberta-with-para/checkpoint-480/model.safetensors
/kaggle/input/roberta-with-para/checkpoint-480/optimizer.pt
/kaggle/input/roberta-with-para/checkpoint-480/rng_state.pth
/kaggle/input/roberta-with-para/checkpoint-160/config.json
/kaggle/input/roberta-with-para/checkpoint-160/trainer_state.json
/kaggle/input/roberta-with-para/checkpoint-160/training_args.bin
/kaggle/input/roberta-with-para/checkpoint-160/scheduler.pt
/kaggle/input/roberta-with-para/checkpoint-160/model.saf

In [2]:
! pip install accelarate -U
! pip install datasets

ERROR: Could not find a version that satisfies the requirement accelarate (from versions: none)
ERROR: No matching distribution found for accelarate


In [3]:
import wandb
api_key = "9963cf6219e451d47251ea34645181ada1b2526b"
wandb.login(key=api_key)
wandb.init()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: raghav21274 (ragha). Use `wandb login --relogin` to force relogin


In [4]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    DebertaTokenizer,
    DebertaForSequenceClassification,
    AutoConfig,
)

import matplotlib.pyplot as plt

import csv
from torch.utils.data import DataLoader, Dataset
import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    print('Using CPU')

PATH="/kaggle/input/webis-clickbait-22/"
OUTPATH="/kaggle/working/"
# PATH = "D:\\ghd\\NLP-Project\\webis-clickbait-22\\"
# OUTPATH = "D:\\ghd\\NLP-Project\\output\\"
BATCH=8
sep='[SEP]'

label_encoder = LabelEncoder()

roberta = "roberta-base"
deberta = "microsoft/deberta-base"

2024-04-25 08:36:36.680730: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 08:36:36.680843: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 08:36:36.812840: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using GPU: Tesla P100-PCIE-16GB


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer

class Dataset(torch.utils.data.Dataset):
    def __init__(self,encodings,labels=None):
        self.labels = labels
        self.encodings = encodings
    
    def __getitem__(self,idx):
        item = {key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
        if self.labels is not None:
            item["label"]  = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [6]:
data_train=pd.read_json("/kaggle/input/webis-clickbait-22/train.jsonl",lines=True,encoding='utf-8')
data_test=pd.read_json("/kaggle/input/webis-clickbait-22/test.jsonl",lines=True,encoding='utf-8')
data_val=pd.read_json("/kaggle/input/webis-clickbait-22/validation.jsonl",lines=True,encoding='utf-8')


In [7]:
def get_split_data(data,if_train,option):
    global label_encoder
#     data=pd.read_json(file_path,lines=True,encoding='utf-8')

    y = data["tags"]
    Y = [i[0] for i in y]
    print("len of Y: ",len(Y))
    if if_train:
        Y = label_encoder.fit_transform(Y)
    else:
        Y = label_encoder.transform(Y)

    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords']]
    dic=new_data.to_dict(orient='records')

    desccount=0
    keycount=0
    for i in dic:
        # print(i)
        i['postText']='postText: '+i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']='targetParagraphs: '+targetparah
        i['targetTitle']='targetTitle: '+i['targetTitle']
        targetDescription=i['targetDescription']
        if(targetDescription is None or len(targetDescription)==0):
            targetDescription='none'
            desccount+=1
        i['targetDescription']='targetDescription: '+targetDescription
        targetkey=i['targetKeywords']
        if(targetkey is None or len(targetkey)==0):
            targetkey='none'
            keycount+=1
        i['targetKeywords']='targetKeywords: '+targetkey
        if option==1:
            i['text']=i['postText']+" "+i['targetTitle']+" "+i['targetKeywords']+" "+sep+i['targetDescription']
        else:
            i['text']=i['targetParagraphs']+sep+i['postText']
        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']

    print("desccount: ",desccount)
    print("keycount: ",keycount)
    X = [i['text'] for i in dic]
    X=np.array(X)
#     return X[0:100],Y[0:100]
    return X,Y

In [8]:
x_train_without_para,y_train = get_split_data(data_train,True,1)
x_test_without_para,y_test = get_split_data(data_test,False,1)
x_val_without_para,y_val= get_split_data(data_val,False,1)

len of Y:  3200
desccount:  332
keycount:  1314
len of Y:  1000
desccount:  104
keycount:  414
len of Y:  800
desccount:  78
keycount:  330


In [9]:
x_train_with_para,y_train = get_split_data(data_train,True,0)
x_test_with_para,y_test = get_split_data(data_test,False,0)
x_val_with_para,y_val= get_split_data(data_val,False,0)

len of Y:  3200
desccount:  332
keycount:  1314
len of Y:  1000
desccount:  104
keycount:  414
len of Y:  800
desccount:  78
keycount:  330


In [10]:
Roberta_tokenizer_without_para=RobertaTokenizerFast.from_pretrained('/kaggle/input/roberta-without-para/roberta_without_para')
Deberta_tokenizer_without_para = DebertaTokenizer.from_pretrained("/kaggle/input/deberta-0-66/deberta_overall")

In [11]:
Roberta_tokenizer_with_para=RobertaTokenizerFast.from_pretrained('/kaggle/input/roberta-with-para/roberta_overall')
Deberta_tokenizer_with_para = DebertaTokenizer.from_pretrained("/kaggle/input/dberta-para/deberta_overall")

In [12]:
num=len(label_encoder.classes_)
id_to_label = {int(idd):label for label, idd in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}
print("ID to Label mapping:", id_to_label)
label_to_id  = {label:int(idd) for label, idd in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}
print("Label to ID mapping:", label_to_id)

ID to Label mapping: {0: 'multi', 1: 'passage', 2: 'phrase'}
Label to ID mapping: {'multi': 0, 'passage': 1, 'phrase': 2}


In [13]:
# config =AutoConfig.from_pretrained(roberta)
# config.update({"id2label": id_to_label})
roberta_model_without_para = RobertaForSequenceClassification.from_pretrained('/kaggle/input/roberta-without-para/roberta_without_para')
roberta_model_with_para = RobertaForSequenceClassification.from_pretrained('/kaggle/input/roberta-with-para/roberta_overall')
# dconfig =AutoConfig.from_pretrained(deberta)
# dconfig.update({"id2label": id_to_label})
# dconfig.update({"label2id":label_to_id})
# dconfig.update({"num_labels":3})
deberta_model_without_para = DebertaForSequenceClassification.from_pretrained('/kaggle/input/deberta-0-66/deberta_overall')
deberta_model_with_para = DebertaForSequenceClassification.from_pretrained('/kaggle/input/dberta-para/deberta_overall')

In [14]:
roberta_model_without_para.to(device)
roberta_model_with_para.to(device)

deberta_model_without_para.to(device)
deberta_model_with_para.to(device)

DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (

In [15]:
from tqdm import tqdm

y_pred_roberta_train_without_para=[]
for i in tqdm(x_train_without_para, desc="RoBERTa Training without para"):
    text = i
    input_ids = Roberta_tokenizer_without_para(text, padding=True, truncation=True, return_tensors="pt")['input_ids']
    input_ids = input_ids.to(device)
    with torch.no_grad():
        output = roberta_model_without_para(input_ids)
        y_pred_roberta_train_without_para.append(output["logits"].cpu().numpy())
y_pred_roberta_train_without_para=np.array(y_pred_roberta_train_without_para)
np.save("roberta_train_without_para.npy",y_pred_roberta_train_without_para)

y_pred_roberta_val_without_para=[]
for i in tqdm(x_val_without_para, desc="RoBERTa validation without para"):
    text = i
    input_ids = Roberta_tokenizer_without_para(text, padding=True, truncation=True, return_tensors="pt")['input_ids']
    input_ids = input_ids.to(device)
    with torch.no_grad():
        output = roberta_model_without_para(input_ids)
        y_pred_roberta_val_without_para.append(output["logits"].cpu().numpy())
y_pred_roberta_val_without_para=np.array(y_pred_roberta_val_without_para)
np.save("roberta_val_without_para.npy",y_pred_roberta_val_without_para)



y_pred_roberta_test_without_para=[]
for i in tqdm(x_test_without_para, desc="RoBERTa testing without para"):
    text = i
    input_ids = Roberta_tokenizer_without_para(text, padding=True, truncation=True, return_tensors="pt")['input_ids']
    input_ids = input_ids.to(device)
    with torch.no_grad():
        output = roberta_model_without_para(input_ids)
        y_pred_roberta_test_without_para.append(output["logits"].cpu().numpy())
y_pred_roberta_test_without_para=np.array(y_pred_roberta_test_without_para)
np.save("roberta_test_without_para.npy",y_pred_roberta_test_without_para)

RoBERTa testing without para: 100%|██████████| 1000/1000 [00:10<00:00, 92.12it/s]


In [16]:
y_pred_roberta_train_with_para=[]
for i in tqdm(x_train_with_para, desc="RoBERTa Training with para"):
    text = i
    input_ids = Roberta_tokenizer_with_para(text, padding=True, truncation=True, return_tensors="pt")['input_ids']
    input_ids = input_ids.to(device)
    with torch.no_grad():
        output = roberta_model_with_para(input_ids)
        y_pred_roberta_train_with_para.append(output["logits"].cpu().numpy())
y_pred_roberta_train_with_para=np.array(y_pred_roberta_train_with_para)
np.save("roberta_train_with_para.npy",y_pred_roberta_train_with_para)


y_pred_roberta_val_with_para=[]
for i in tqdm(x_val_with_para, desc="RoBERTa validation with para"):
    text = i
    input_ids = Roberta_tokenizer_with_para(text, padding=True, truncation=True, return_tensors="pt")['input_ids']
    input_ids = input_ids.to(device)
    with torch.no_grad():
        output = roberta_model_with_para(input_ids)
        y_pred_roberta_val_with_para.append(output["logits"].cpu().numpy())
y_pred_roberta_val_with_para=np.array(y_pred_roberta_val_with_para)
np.save("roberta_val_with_para.npy",y_pred_roberta_val_with_para)

y_pred_roberta_test_with_para=[]
for i in tqdm(x_test_with_para, desc="RoBERTa testing with para"):
    text = i
    input_ids = Roberta_tokenizer_with_para(text, padding=True, truncation=True, return_tensors="pt")['input_ids']
    input_ids = input_ids.to(device)
    with torch.no_grad():
        output = roberta_model_with_para(input_ids)
        y_pred_roberta_test_with_para.append(output["logits"].cpu().numpy())

y_pred_roberta_test_with_para=np.array(y_pred_roberta_test_with_para)
np.save("roberta_test_with_para.npy",y_pred_roberta_test_with_para)

RoBERTa testing with para: 100%|██████████| 1000/1000 [00:21<00:00, 45.90it/s]


In [17]:
y_pred_deberta_train_without_para=[]
for i in tqdm(x_train_without_para, desc="DeBERTa Training without para"):
    text = i
    input_ids = Deberta_tokenizer_without_para(text, padding=True, truncation=True, return_tensors="pt")['input_ids']
    input_ids = input_ids.to(device)
    with torch.no_grad():
        output = deberta_model_without_para(input_ids)
        y_pred_deberta_train_without_para.append(output["logits"].cpu().numpy())
y_pred_deberta_train_without_para=np.array(y_pred_deberta_train_without_para)
np.save("deberta_train_without_para.npy",y_pred_deberta_train_without_para)

y_pred_deberta_val_without_para=[]
for i in tqdm(x_val_without_para, desc="DeBERTa validation without para"):
    text = i
    input_ids = Deberta_tokenizer_without_para(text, padding=True, truncation=True, return_tensors="pt")['input_ids']
    input_ids = input_ids.to(device)
    with torch.no_grad():
        output = deberta_model_without_para(input_ids)
        y_pred_deberta_val_without_para.append(output["logits"].cpu().numpy())
y_pred_deberta_val_without_para=np.array(y_pred_deberta_val_without_para)
np.save("deberta_val_without_para.npy",y_pred_deberta_val_without_para)


y_pred_deberta_test_without_para=[]
for i in tqdm(x_test_without_para, desc="DeBERTa testing without para"):
    text = i
    input_ids = Deberta_tokenizer_without_para(text, padding=True, truncation=True, return_tensors="pt")['input_ids']
    input_ids = input_ids.to(device)
    with torch.no_grad():
        output = deberta_model_without_para(input_ids)
        y_pred_deberta_test_without_para.append(output["logits"].cpu().numpy())
y_pred_deberta_test_without_para=np.array(y_pred_deberta_test_without_para)
np.save("deberta_test_without_para.npy",y_pred_deberta_test_without_para)

DeBERTa testing without para: 100%|██████████| 1000/1000 [00:23<00:00, 42.27it/s]


In [18]:
y_pred_deberta_train_with_para=[]
for i in tqdm(x_train_with_para, desc="DeBerta Training with para"):
    text = i
    input_ids = Deberta_tokenizer_with_para(text, padding=True, truncation=True, return_tensors="pt")['input_ids']
    input_ids = input_ids.to(device)
    with torch.no_grad():
        output = deberta_model_with_para(input_ids)
        y_pred_deberta_train_with_para.append(output["logits"].cpu().numpy())
y_pred_deberta_train_with_para=np.array(y_pred_deberta_train_with_para)
np.save("deberta_train_with_para.npy",y_pred_deberta_train_with_para)


y_pred_deberta_val_with_para=[]
for i in tqdm(x_val_with_para, desc="DeBERTa Validation with para"):
    text = i
    input_ids = Deberta_tokenizer_with_para(text, padding=True, truncation=True, return_tensors="pt")['input_ids']
    input_ids = input_ids.to(device)
    with torch.no_grad():
        output = deberta_model_with_para(input_ids)
        y_pred_deberta_val_with_para.append(output["logits"].cpu().numpy())
y_pred_deberta_val_with_para=np.array(y_pred_deberta_val_with_para)
np.save("deberta_val_with_para.npy",y_pred_deberta_val_with_para)
        

y_pred_deberta_test_with_para=[]
for i in tqdm(x_test_with_para, desc="DeBERTa testing with para"):
    text = i
    input_ids = Deberta_tokenizer_with_para(text, padding=True, truncation=True, return_tensors="pt")['input_ids']
    input_ids = input_ids.to(device)
    with torch.no_grad():
        output = deberta_model_with_para(input_ids)
        y_pred_deberta_test_with_para.append(output["logits"].cpu().numpy())
y_pred_deberta_test_with_para=np.array(y_pred_deberta_test_with_para)
np.save("deberta_test_with_para.npy",y_pred_deberta_test_with_para)

DeBERTa testing with para: 100%|██████████| 1000/1000 [00:38<00:00, 26.20it/s]


In [19]:
y_pred_roberta_train_without_para.shape,y_pred_roberta_val_without_para.shape,y_pred_roberta_test_without_para.shape,y_pred_roberta_train_with_para.shape,y_pred_roberta_val_with_para.shape,y_pred_roberta_test_with_para.shape


((3200, 1, 3),
 (800, 1, 3),
 (1000, 1, 3),
 (3200, 1, 3),
 (800, 1, 3),
 (1000, 1, 3))

In [20]:
y_pred_deberta_train_without_para.shape,y_pred_deberta_val_without_para.shape,y_pred_deberta_test_without_para.shape,y_pred_deberta_train_with_para.shape,y_pred_deberta_val_with_para.shape,y_pred_deberta_test_with_para.shape

((3200, 1, 3),
 (800, 1, 3),
 (1000, 1, 3),
 (3200, 1, 3),
 (800, 1, 3),
 (1000, 1, 3))